In [14]:
import pandas as pd
from ggplot import *
import json
%matplotlib inline

In [15]:
rawdata = pd.read_csv("Resources_Pesticides_Use_E_All_Data.csv")[["Country", "Year", "Item", "Value"]]
cleandata = rawdata.fillna(0)
cleandata["Item"]
topchem = cleandata.groupby(["Item"], as_index=0)["Value"].sum().sort_values(by="Value", ascending=0)
topchem = topchem[-topchem["Item"].str.contains("Other")][-topchem["Item"].str.contains("Total")][:25]
cleandata = cleandata[cleandata["Item"].isin(topchem["Item"])]
#cleamdata =  cleandata[cleandata["Item"].isin(["Amides", "Inorganics", "Dinitroanilines", "Dithiocarbamates", ""])
alldata = cleandata.groupby(["Country", "Item", "Year"], as_index=0).sum().sort_values(by="Country")

In [16]:
def fdrec(df):
    drec = dict()
    ncols = df.values.shape[1]
    for line in df.values:
        d = drec
        for j, col in enumerate(line[:-1]):
            if not col in d.keys():
                if j != ncols-2:
                    d[col] = {}
                    d = d[col]
                else:
                    d[col] = line[-1]
            else:
                if j!= ncols-2:
                    d = d[col]
    return drec
with open("pesticide.json", "w") as datafile:
    datadict = fdrec(alldata)
    json.dump(datadict, datafile, ensure_ascii=False)

In [17]:
#Pesticide Plot (All)
chemsperyear = cleandata.groupby(["Item", "Year"], as_index=0)["Value"].sum()
#ggplot(aes(x="Year", y="Value", color="Item"), data=chemsperyear)+geom_line()
with open("chemsperyear.json", "w") as datafile:
    datadict = fdrec(chemsperyear)
    json.dump(datadict, datafile, ensure_ascii=False)

In [18]:
chemspercountry = cleandata.groupby(["Item", "Country"], as_index=0)["Value"].sum()
with open("chemspercountry.json", "w") as datafile:
    datadict = fdrec(chemspercountry)
    json.dump(datadict, datafile, ensure_ascii=False)

In [19]:
#DDT
ddtraw = rawdata.loc[rawdata["Item"]=="Chlorinated Hydrocarbons"]
ddtbyyear = ddtraw.groupby(["Year", "Country"], as_index=0)["Value"].sum()
#ggplot(aes(x="Year", y="Value", color="Country"), data=ddtbyyear.)+geom_line()

In [ ]:
ddtbycountry = ddtraw.groupby("Country", as_index=0)["Value"].sum().sort_values(by="Value", ascending=0)
top10 = ddtbycountry.iloc[1:10, 0]
top10

In [ ]:
selectddtbyyear = ddtbyyear.ix[ddtbyyear["Country"].isin(top10), :]
ggplot(aes(x="Year", y="Value", color="Country"), data=selectddtbyyear)+geom_line()